In [ ]:
!pip install tensorflow==2.14.0

In [ ]:
import numpy as np
import tensorflow as tf

# tf.enable_eager_execution()
print(tf.__version__)

2.14.0


In [ ]:
Users = ['Rizki','Misel','Via','Hasan']
ProdukUtama = ['Bakso 99','Toko Sayuran Bu Umi','Foto Copy NN','Butik Indah','Meubel Raharja','Agro Blimbing']
Umkms = ['kuliner','otomotif','fashion','Agrobisnis','produk digital']

num_users = len(Users)
num_produk_utama = len(ProdukUtama)
num_umkms = len(Umkms)
num_recomendations = 4

In [ ]:
Users_ProdukUtama = tf.constant([
                [4, 6, 8, 0, 0, 0],
                [0, 0, 10, 0, 8, 3],
                [0, 6, 0, 0, 3, 7],
                [10, 9, 0, 5, 0, 2]],dtype=tf.float32)

# one-hot encoded
ProdukUtama_Umkms = tf.constant([
                [1, 1, 0, 0, 1],
                [1, 1, 0, 0, 0],
                [0, 0, 1, 1, 0],
                [1, 0, 1, 1, 0],
                [0, 0, 0, 0, 1],
                [1, 0, 0, 0, 1]],dtype=tf.float32)

In [ ]:
users_umkms = tf.matmul(Users_ProdukUtama,ProdukUtama_Umkms)
users_umkms

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[10., 10.,  8.,  8.,  4.],
       [ 3.,  0., 10., 10., 11.],
       [13.,  6.,  0.,  0., 10.],
       [26., 19.,  5.,  5., 12.]], dtype=float32)>

In [ ]:
users_umkms = users_umkms/tf.reduce_sum(users_umkms,axis=1,keepdims=True)
users_umkms


<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[0.25      , 0.25      , 0.2       , 0.2       , 0.1       ],
       [0.0882353 , 0.        , 0.29411766, 0.29411766, 0.32352942],
       [0.44827586, 0.20689656, 0.        , 0.        , 0.3448276 ],
       [0.3880597 , 0.2835821 , 0.07462686, 0.07462686, 0.17910448]],
      dtype=float32)>

In [ ]:
top_users_umkms = tf.nn.top_k(users_umkms,num_umkms)[1]
top_users_umkms

<tf.Tensor: shape=(4, 5), dtype=int32, numpy=
array([[0, 1, 2, 3, 4],
       [4, 2, 3, 0, 1],
       [0, 4, 1, 2, 3],
       [0, 1, 4, 2, 3]], dtype=int32)>

In [ ]:
for i in range(num_users) :
    Umkms_names = [Umkms[int(index)] for index in top_users_umkms[i]]
    print('{}: {}'.format(Users[i],Umkms_names))

Rizki: ['kuliner', 'otomotif', 'fashion', 'Agrobisnis', 'produk digital']
Misel: ['produk digital', 'fashion', 'Agrobisnis', 'kuliner', 'otomotif']
Via: ['kuliner', 'produk digital', 'otomotif', 'fashion', 'Agrobisnis']
Hasan: ['kuliner', 'otomotif', 'produk digital', 'fashion', 'Agrobisnis']


In [ ]:
users_ratings = tf.matmul(users_umkms,tf.transpose(ProdukUtama_Umkms))
users_ratings

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.6       , 0.5       , 0.4       , 0.65      , 0.1       ,
        0.35      ],
       [0.4117647 , 0.0882353 , 0.5882353 , 0.67647064, 0.32352942,
        0.4117647 ],
       [1.        , 0.6551724 , 0.        , 0.44827586, 0.3448276 ,
        0.79310346],
       [0.8507463 , 0.6716418 , 0.14925373, 0.53731346, 0.17910448,
        0.5671642 ]], dtype=float32)>

In [ ]:
users_ratings_new = tf.where(tf.equal(Users_ProdukUtama, tf.zeros_like(Users_ProdukUtama)),
                                  users_ratings,
                             tf.zeros_like(tf.cast(Users_ProdukUtama, tf.float32)))
users_ratings_new

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.65      , 0.1       ,
        0.35      ],
       [0.4117647 , 0.0882353 , 0.        , 0.67647064, 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , 0.44827586, 0.        ,
        0.        ],
       [0.        , 0.        , 0.14925373, 0.        , 0.17910448,
        0.        ]], dtype=float32)>

In [ ]:
top_ProdukUtama = tf.nn.top_k(users_ratings_new,num_recomendations)[1]
top_ProdukUtama

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[3, 5],
       [3, 0],
       [0, 3],
       [4, 2]], dtype=int32)>

In [ ]:
for i in range(num_users):
    ProdukUtama_names = [ProdukUtama[index] for index in top_ProdukUtama[i]]
    print('{}: {}'.format(Users[i],ProdukUtama_names))


Rizki: ['Butik Indah', 'Agro Blimbing']
Misel: ['Butik Indah', 'Bakso 99']
Via: ['Bakso 99', 'Butik Indah']
Hasan: ['Meubel Raharja', 'Foto Copy NN']
